In [1]:
from chicksexer import predict_gender
import pandas as pd

In [2]:



file_path = "../dataset/predicted_stances/updated_predicted_stances.csv"

import os
print(os.path.exists(file_path))


output_dir = "../dataset/predicted_genders/"
output_path = os.path.join(output_dir, "predicted_genders_87.csv")


if not os.path.exists(file_path):
    raise FileNotFoundError("Input file not found at: {}".format(file_path))


True


In [3]:
def clean_name(name):
    """Remove invalid characters like '_' from the name."""
    return name.replace("_", "")


prediction_cache = {}

try:
    
    data = pd.read_csv(file_path, engine='python')

    
    if 'user' not in data.columns:
        raise KeyError("'user' column not found in the input file.")

  
    genders = []
    for name in data['user']:
        if pd.notna(name):  
            cleaned_name = clean_name(name)  
            
            
            if cleaned_name in prediction_cache:
                genders.append(prediction_cache[cleaned_name])
            else:
                try:
                    prediction = predict_gender(cleaned_name, return_proba=True)
                    
                    
                    gender, prob = max(prediction.items(), key=lambda x: x[1])
                    
                    
                    if prob > 0.87:
                        prediction_cache[cleaned_name] = gender
                    else:
                        prediction_cache[cleaned_name] = 'unknown'

                    genders.append(prediction_cache[cleaned_name])
                except Exception as e:
                    prediction_cache[cleaned_name] = "Error: {}".format(e)
                    genders.append(prediction_cache[cleaned_name])
        else:
            genders.append('unknown')

   
    data['gender'] = genders

   
    data.to_csv(output_path, index=False)
    print("Processing complete. Updated file saved to {}".format(output_path))

except Exception as e:
    print("Error: {}".format(e))

2025-01-02 20:41:20,121 - chicksexer.api - INFO - Loading model (only required for the initial prediction)...


KeyboardInterrupt: 

In [5]:
from collections import Counter

gender_counts = Counter(data['gender'])
print("Unique users per gender:")
for gender, count in gender_counts.items():
    print("{}: {}".format(gender, count))

Unique users per gender:
unknown: 30697
female: 24015
male: 49830


In [6]:
data = pd.read_csv(output_path)

unique_users = data.drop_duplicates(subset='user')


unique_gender_counts = unique_users['gender'].value_counts()


gender_table = pd.DataFrame({
    'Gender': unique_gender_counts.index,
    'Count': unique_gender_counts.values
})

print(gender_table)

   Count   Gender
0   9424     male
1   5503  unknown
2   4209   female
